In [13]:
!pip install -q lxml

In [44]:
#!pip install -q lxml

import bs4
import lxml
import pandas
import urllib
from tqdm.auto import tqdm

from urllib import request

In [45]:
""" Création du dictionnaire Racine """

data_all = {
    i : {j : {} for j in range(1,13)} for i in range(2011,2019)
}

In [ ]:
""" Scrapping du site infoclimat.fr et organisation des données sous forme de dictionnaire """
# Il faut optimiser la création en ne créant qu'une seule fois le Df lié aux villes

for i in tqdm(range(len(data_all))):
    for y in data_all :
        for m in data_all[y] :
            url_temp = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
            raw_text = request.urlopen(url_temp).read()
            page = bs4.BeautifulSoup(raw_text) #,'lxml')
            tableau = page.find('table', {'id' : 'tableau-releves'})
            rows = tableau.find_all('tr')
    
            df = {
        'Villes' : [rows[i].find('a').text for i in range(1,len(rows))], #[rows[i].find_all('div')[0].text for i in range(1,len(rows))], #[rows[i].find('a').text for i in range(1,len(rows))],
        'Température minimale extrême du mois' : [rows[i].find('div').text for i in range(1,len(rows))],
        'Moyenne des températures minimales du mois' : [rows[i].find_all('td')[2].text for i in range(1,len(rows))],
        'Température moyenne du mois' : [rows[i].find_all('td')[3].text for i in range(1,len(rows))],
        'Moyenne des températures maximales du mois' : [rows[i].find_all('td')[4].text for i in range(1,len(rows))],
        'Température maximale extrême du mois' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows))],
        'Cumul de précipitation du mois' : [rows[i].find_all('td')[6].text for i in range(1,len(rows))],
        "Heure d'ensolleiment du mois" : [rows[i].find_all('td')[7].text for i in range(1,len(rows))],
        'Rafale maximale du mois' : [rows[i].find_all('td')[8].text for i in range(1,len(rows))] 
                }
    
            data_all[y][m] = df

In [173]:
""" Conversion en DataFrame pour exporter """

df_all = pandas.DataFrame(data_all)

In [ ]:
""" Exportation au format json """

df_all.to_json('data_base.json')

In [ ]:
""" Normalisation des variables suivant la nomenclature de la source """

df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensolleiment du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)

In [8]:
""" Définition de la classe finale utilisée """

class Meteo :
    """
    Class permettant de scrapper et d'effectuer des opérations sur les données d'infoclimat concernant la température observée par différentes 
    stations météo entre 2011 et 2018.
    """
    def __init__(self):
        """
        Initialisation de la class contenant :
            self.data_all : dict
                Contient l'ensemble des données par année (initialement vides)
        """
        self.data_all = {
                i : {j : {} for j in range(1,13)} for i in range(2011,2019)
            }

    def scrap(self):
        """
        Fonction permettant de scrapper les données voulues, i.e. les données d'infoclimat concernant la température observée par différentes 
        stations météo entre 2011 et 2018

            Paramètres:
            ------------
                self.data_all : dict
                    Dictionnaire contenant toutes les données par années
        
            Output:
            ----------
                self.data_pop : dict
                    Dictionnaire contenant toutes les données par années actualisées
            
        """
        for y in self.data_all :
            for m in self.data_all[y] :
                url_temp = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
                raw_text = request.urlopen(url_temp).read()
                page = bs4.BeautifulSoup(raw_text,'lxml')
                tableau = page.find('table', {'id' : 'tableau-releves'})
                rows = tableau.find_all('tr')
        
                df = {
            'Villes' : [rows[i].find('a').text for i in range(1,len(rows))], #[rows[i].find_all('div')[0].text for i in range(1,len(rows))], #[rows[i].find('a').text for i in range(1,len(rows))],
            'Température minimale extrême du mois' : [rows[i].find('div').text for i in range(1,len(rows))],
            'Moyenne des températures minimales du mois' : [rows[i].find_all('td')[2].text for i in range(1,len(rows))],
            'Température moyenne du mois' : [rows[i].find_all('td')[3].text for i in range(1,len(rows))],
            'Moyenne des températures maximales du mois' : [rows[i].find_all('td')[4].text for i in range(1,len(rows))],
            'Température maximale extrême du mois' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows))],
            'Cumul de précipitation du mois' : [rows[i].find_all('td')[6].text for i in range(1,len(rows))],
            "Heure d'ensolleiment du mois" : [rows[i].find_all('td')[7].text for i in range(1,len(rows))],
            'Rafale maximale du mois' : [rows[i].find_all('td')[8].text for i in range(1,len(rows))] 
                    }
        
                self.data_all[y][m] = df

    def export(self):
        """
        Fonction intégrée d'export pour permettant de faciliter les manipulations.

        Paramètres:
        ------------
            self.data_pop : dict
                Dictionnaire contenant toutes les données par années
        
        Output:
        ----------
            Aucun
        """
        
        self.df_all = pandas.DataFrame(self.data_all)
        self.df_all.to_json('data_base.json')

    def rename(self):
        """
        Fonction intégrée permettant de renommer facilement les colonnes par le nom utilisé par infoclimat dans le but de faciliter le traitement.

        Paramètres :
        ------------
            self.df_all : DataFrame
                DataFrame contenant toutes les données météo extraites

        Output :
        --------
            self.df_all : DataFrame
                DataFrame contenant les mêmes données mais dont les colonnes ont été renommées
        """
        
        self.df_all = pandas.DataFrame(self.data_all)
        self.df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensolleiment du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)
    